In [154]:
import random as rd
import simpy
import pandas as pd
from datetime import datetime

In [155]:
df = pd.read_csv("simulated_call_centre.csv")
#df.sample(10,random_state=40)

In [156]:
def convert_to_seconds(time_string):
    time_obj = datetime.strptime(time_string, "%I:%M:%S %p")
    time_in_seconds = (time_obj.hour * 3600) + (time_obj.minute * 60) + time_obj.second
    return time_in_seconds

def subtract(time_seconds):
    return time_seconds - 28800

In [157]:
day_string = df.groupby('date')['wait_length'].sum().idxmax()
df_day = df[df["date"] == day_string]

df_day['call_started'] = df_day['call_started'].apply(convert_to_seconds)
df_day['call_started'] = df_day['call_started'].apply(subtract)
df_day.drop(columns=["date","daily_caller","call_answered","call_ended"])

C:\Users\Camilo\AppData\Local\Temp\ipykernel_14632\3135047881.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_day['call_started'] = df_day['call_started'].apply(convert_to_seconds)
C:\Users\Camilo\AppData\Local\Temp\ipykernel_14632\3135047881.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_day['call_started'] = df_day['call_started'].apply(subtract)


,call_id,call_started,wait_length,service_length,meets_standard
47359,47360,0,0,579,True
47360,47361,139,0,253,True
47361,47362,174,0,693,True
47362,47363,398,0,68,True
47363,47364,560,0,109,True
...,...,...,...,...,...
47651,47652,35277,14,385,True
47652,47653,35518,0,409,True
47653,47654,35535,0,17,True
47654,47655,35543,10,41,True


In [158]:
start = df_day['call_started'].tolist()
lenght = df_day['service_length'].tolist()
start_length = zip(start,lenght)

## Simulação Inicial

In [159]:
#variáveis da simulação
SEED = True
RANDOM_SEED = 616
NUM_ATENDENTES = 1
SIM_TIME = 10000000
CLIENT_LIMIT = True
MAX_CLIENT = 20

In [160]:
class CallCentre:

    def __init__(self,env,num_atendentes):
        self.env = env
        self.atendente = simpy.Resource(env,num_atendentes)
        self.espera = []

    def atendimento(self,execution_time): # executa o tempo de atendimento

        yield self.env.timeout(execution_time)
        #print(f"Cliente {cliente} é atendido as {self.env.now}")

def cliente(env,nome,execution_time,cc): # gera um cliente

    with cc.atendente.request() as request:
        atendimento_solicitado = env.now
        print(f"Cliente {nome} solicita atendimento as {atendimento_solicitado}")
        
        yield request
        atendimento_iniciado = env.now
        cc.espera.append(str(atendimento_iniciado - atendimento_solicitado))
        print(f"Cliente {nome} inicia atendimento as {atendimento_iniciado}")

        yield env.process(cc.atendimento(execution_time))
        print(f"Cliente {nome} desliga as {env.now}")

"""
def run(env,cc,client_limit,max_client): # executa a simulação
    nome = 1

    env.process(cliente(env,nome,cc))

    while True:
        yield env.timeout(rd.randint(2,10)) # definir e variar valor
        nome += 1
        if client_limit:
            if nome <= max_client: env.process(cliente(env,nome,cc))
        else:
            env.process(cliente(env,nome,cc))
"""

def run(env,cc): # executa a simulação
    nome = 0
    #env.process(cliente(env,nome,cc))

    for start_time,execution_time in start_length:
        yield env.timeout(start_time)
        nome += 1
        env.process(cliente(env,nome,execution_time,cc))

In [161]:
if SEED: rd.seed(RANDOM_SEED)

env = simpy.Environment()
call_centre = CallCentre(env,NUM_ATENDENTES)

#espera = env.process(run(env,call_centre,CLIENT_LIMIT,MAX_CLIENT))
espera = env.process(run(env,call_centre))


print("saída:")
env.run(until=SIM_TIME)

saída:
Cliente 1 solicita atendimento as 0
Cliente 1 inicia atendimento as 0
Cliente 2 solicita atendimento as 139
Cliente 3 solicita atendimento as 313
Cliente 1 desliga as 579
Cliente 2 inicia atendimento as 579
Cliente 4 solicita atendimento as 711
Cliente 2 desliga as 832
Cliente 3 inicia atendimento as 832
Cliente 5 solicita atendimento as 1271
Cliente 3 desliga as 1525
Cliente 4 inicia atendimento as 1525
Cliente 4 desliga as 1593
Cliente 5 inicia atendimento as 1593
Cliente 5 desliga as 1702
Cliente 6 solicita atendimento as 1858
Cliente 6 inicia atendimento as 1858
Cliente 6 desliga as 1975
Cliente 7 solicita atendimento as 3166
Cliente 7 inicia atendimento as 3166
Cliente 7 desliga as 3399
Cliente 8 solicita atendimento as 4532
Cliente 8 inicia atendimento as 4532
Cliente 8 desliga as 4758
Cliente 9 solicita atendimento as 5919
Cliente 9 inicia atendimento as 5919
Cliente 9 desliga as 6116
Cliente 10 solicita atendimento as 7436
Cliente 10 inicia atendimento as 7436
Cliente 10

In [162]:
wait = df_day['wait_length'].tolist()

#print(len(wait),len(call_centre.espera))

for i in range(len(call_centre.espera)):
    if int(call_centre.espera[i]) != 0:
        print(call_centre.espera[i],wait[i])
    

440 0
519 0
814 0
322 0


## Simulação com Melhoria